In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import talib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os
from sklearn.model_selection import TimeSeriesSplit
#import eli5
#from eli5.sklearn import PermutationImportance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.trial import TrialState
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import plotly.graph_objs as go
import plotly.offline as pyo
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F
from tqdm.notebook import tqdm


from sklearn.model_selection import KFold
import copy
from torch.cuda.amp import GradScaler, autocast
import os


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())


gpu
2.1.2+cu121
CUDA available: True
CUDA version: 12.1
cuDNN version: 8902


In [3]:
pivoted_data = pd.read_csv("../Data/longmerged_deneme_51.csv")
pivoted_data.drop(columns=["Unnamed: 0"], inplace=True)
#pivoted_data.set_index("PENDS", inplace=True)
len(pivoted_data["OFTIC"].unique())
# pivoted_data[pivoted_data["OFTIC"] == "AAPL"]
pivoted_data = pivoted_data[pivoted_data['PENDS'] > '2011-03-31']
pivoted_data = pivoted_data[pivoted_data['PENDS'] < '2020-01-01']

pivoted_data

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
0,2011-06-30,AAPL,2011-06-30,17.869831,1.698451,2.6718,0.422900,777.000000,0.209015,0.00,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
1,2011-09-30,AAPL,2011-09-30,17.834875,3.131120,2.9872,0.397600,1645.000000,0.258659,0.00,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
2,2011-12-31,AAPL,2011-12-31,21.771714,3.087308,3.4500,0.665700,1321.000000,0.249095,0.00,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
3,2012-03-31,AAPL,2012-03-31,26.426070,2.380069,3.9154,0.528200,1457.000000,0.153558,0.00,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
4,2012-06-30,AAPL,2012-06-30,27.652509,3.109565,4.2582,0.384300,2056.000000,0.204900,0.00,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2018-12-31,ZION,2018-12-31,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.30,...,0.355782,-15.534891,-5.840908,-24.280433,-13.633068,-9.080399,-17.844385,-17.722026,-12.791300,0.512814
14056,2019-03-31,ZION,2019-03-31,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.30,...,-0.178828,14.988381,10.496262,15.292074,7.934506,6.057101,16.488114,19.393354,9.857481,9.920635
14057,2019-06-30,ZION,2019-06-30,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.30,...,0.081598,4.699164,3.493137,-3.644893,7.351231,0.970027,3.185392,5.459461,5.405405,2.509890
14058,2019-09-30,ZION,2019-09-30,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.34,...,-0.036101,1.258389,5.768897,-7.078949,1.449274,-2.709415,0.271247,3.190669,-0.512819,8.569506


In [4]:
# pivoted_data = pivoted_data[~pivoted_data["OFTIC"].isin(["DFS", "ICE"])]

In [5]:
print(list(pivoted_data.columns))

['Date', 'OFTIC', 'PENDS', 'MEAN', 'STDEV', 'BPS', 'CPS', 'CPX', 'CSH', 'DPS', 'EBG', 'EBI', 'EBS', 'EBT', 'ENT', 'EPS', 'FFO', 'GPS', 'GRM', 'NAV', 'NDT', 'NET', 'OPR', 'PRE', 'ROA', 'ROE', 'SAL', 'Real_Estate_Index_Price', 'VIX_Close', 'Gold_Close', 'Three_Month_Yield', 'Brent_Close', 'Hrc_close', 'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close', 'Energy_Close', 'Financials_Close', 'Health_care_Close', 'industrials_Close', 'information_Close', 'materials_Close', 'utilities_Close', 'Sector', 'numeric_sector', 'Real_Estate_Index_Price_quarterly_return', 'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return', 'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return', 'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return', 'C_Discretionary_Close_quarterly_return', 'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return', 'Financials_Close_quarterly_return', 'Health_care_Close_quarterly_return', 'industrials_Close_quarterly_ret

In [6]:
unique_counts = pivoted_data.groupby('Sector')['OFTIC'].nunique()
unique_counts

Sector
Communication Services     5
Consumer Discretionary    32
Consumer Staples          20
Energy                    14
Financials                38
Health Care               38
Industrials               36
Information Technology    34
Materials                 19
Real Estate               20
Utilities                 20
Name: OFTIC, dtype: int64

In [7]:
pivoted_data.columns

Index(['Date', 'OFTIC', 'PENDS', 'MEAN', 'STDEV', 'BPS', 'CPS', 'CPX', 'CSH',
       'DPS', 'EBG', 'EBI', 'EBS', 'EBT', 'ENT', 'EPS', 'FFO', 'GPS', 'GRM',
       'NAV', 'NDT', 'NET', 'OPR', 'PRE', 'ROA', 'ROE', 'SAL',
       'Real_Estate_Index_Price', 'VIX_Close', 'Gold_Close',
       'Three_Month_Yield', 'Brent_Close', 'Hrc_close',
       'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close',
       'Energy_Close', 'Financials_Close', 'Health_care_Close',
       'industrials_Close', 'information_Close', 'materials_Close',
       'utilities_Close', 'Sector', 'numeric_sector',
       'Real_Estate_Index_Price_quarterly_return',
       'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return',
       'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return',
       'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return',
       'C_Discretionary_Close_quarterly_return',
       'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return',
   

In [8]:
# pivoted_data.drop(columns=['Real_Estate_Index_Price','Three_Month_Yield', 'Brent_Close', 'Hrc_close',
#        'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close',
#        'Energy_Close', 'Financials_Close', 'Health_care_Close',
#        'industrials_Close', 'information_Close', 'materials_Close',
#        'utilities_Close', 'numeric_sector',
#        'Real_Estate_Index_Price_quarterly_return',
#        'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return',
#        'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return',
#        'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return',
#        'C_Discretionary_Close_quarterly_return',
#        'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return',
#        'Financials_Close_quarterly_return',
#        'Health_care_Close_quarterly_return',
#        'industrials_Close_quarterly_return',
#        'information_Close_quarterly_return',
#        'materials_Close_quarterly_return', 'utilities_Close_quarterly_return'], inplace=True)

In [9]:
#pivoted_data = pivoted_data[pivoted_data["Sector"] == "Financials"]
pivoted_data

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return
0,2011-06-30,AAPL,2011-06-30,17.869831,1.698451,2.6718,0.422900,777.000000,0.209015,0.00,...,0.062957,2.970550,4.378340,-5.517243,-6.345335,7.308964,-1.141483,-1.381423,-1.624192,5.051769
1,2011-09-30,AAPL,2011-09-30,17.834875,3.131120,2.9872,0.397600,1645.000000,0.258659,0.00,...,0.120922,-13.305144,-5.027217,-22.349038,-23.061887,-10.695185,-21.535988,-8.171207,-25.425447,0.418158
2,2011-12-31,AAPL,2011-12-31,21.771714,3.087308,3.4500,0.665700,1321.000000,0.249095,0.00,...,-0.087046,11.933447,9.541476,18.150742,10.076208,9.328708,15.503083,7.838985,14.100815,7.019633
3,2012-03-31,AAPL,2012-03-31,26.426070,2.380069,3.9154,0.528200,1457.000000,0.153558,0.00,...,-0.191850,15.556124,4.893814,3.789965,21.538450,8.417417,10.874069,18.506872,10.358213,-2.612559
4,2012-06-30,AAPL,2012-06-30,27.652509,3.109565,4.2582,0.384300,2056.000000,0.204900,0.00,...,0.484614,-2.905304,2.024644,-7.498254,-7.341768,1.063541,-4.676644,-4.741380,-4.544226,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2018-12-31,ZION,2018-12-31,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.30,...,0.355782,-15.534891,-5.840908,-24.280433,-13.633068,-9.080399,-17.844385,-17.722026,-12.791300,0.512814
14056,2019-03-31,ZION,2019-03-31,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.30,...,-0.178828,14.988381,10.496262,15.292074,7.934506,6.057101,16.488114,19.393354,9.857481,9.920635
14057,2019-06-30,ZION,2019-06-30,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.30,...,0.081598,4.699164,3.493137,-3.644893,7.351231,0.970027,3.185392,5.459461,5.405405,2.509890
14058,2019-09-30,ZION,2019-09-30,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.34,...,-0.036101,1.258389,5.768897,-7.078949,1.449274,-2.709415,0.271247,3.190669,-0.512819,8.569506


In [10]:
unique_pends_by_ticker = pivoted_data.groupby('OFTIC')['PENDS'].nunique()
unique_pends_not_50 = unique_pends_by_ticker[unique_pends_by_ticker != 35]
unique_pends_not_50

Series([], Name: PENDS, dtype: int64)

In [11]:
pivoted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9660 entries, 0 to 14059
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date                                      9660 non-null   object 
 1   OFTIC                                     9660 non-null   object 
 2   PENDS                                     9660 non-null   object 
 3   MEAN                                      9660 non-null   float64
 4   STDEV                                     9660 non-null   float64
 5   BPS                                       9660 non-null   float64
 6   CPS                                       9660 non-null   float64
 7   CPX                                       9660 non-null   float64
 8   CSH                                       9660 non-null   float64
 9   DPS                                       9660 non-null   float64
 10  EBG                                

In [12]:
pivoted_data = pivoted_data.copy()

pivoted_data['Hrc_close'] = pd.to_numeric(pivoted_data['Hrc_close'].astype(str).str.split(",").str[0], errors='coerce')
pivoted_data['Gold_Close'] = pd.to_numeric(pivoted_data['Gold_Close'].astype(str).str.replace(",", ""), errors='coerce').round(2)

In [13]:
company_list = pivoted_data["OFTIC"].unique()
company_list

array(['AAPL', 'AFL', 'ALB', 'AMT', 'AMZN', 'ANSS', 'AON', 'AOS', 'APA',
       'APD', 'APH', 'ARE', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO',
       'BA', 'BDX', 'BEN', 'BG', 'BIIB', 'BK', 'BLK', 'BMY', 'BR', 'BSX',
       'BWA', 'BX', 'CAH', 'CBOE', 'CCI', 'CCL', 'CDNS', 'CE', 'CF',
       'CHD', 'CHRW', 'CHTR', 'CI', 'CMA', 'CMCSA', 'CME', 'CMI', 'CMS',
       'CNC', 'CNP', 'COF', 'COO', 'COST', 'CPB', 'CPRT', 'CPT', 'CRM',
       'CSX', 'CTSH', 'CVS', 'CVX', 'D', 'DAL', 'DE', 'DFS', 'DG', 'DGX',
       'DHI', 'DHR', 'DLR', 'DLTR', 'DOV', 'DPZ', 'DTE', 'DXCM', 'EBAY',
       'ECL', 'ED', 'EFX', 'EIX', 'EMR', 'EOG', 'EQIX', 'EQR', 'ESS',
       'ETN', 'ETR', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FAST',
       'FCX', 'FE', 'FFIV', 'FMC', 'FRT', 'FSLR', 'FTNT', 'GD', 'GILD',
       'GIS', 'GLW', 'GM', 'GNRC', 'GRMN', 'GS', 'GWW', 'HAL', 'HCA',
       'HD', 'HES', 'HIG', 'HOLX', 'HON', 'HPQ', 'HSY', 'HUM', 'IBM',
       'ICE', 'IDXX', 'IEX', 'ILMN', 'INCY', 'IP', 'ISRG', 'IT', 'ITW

In [14]:
# pivoted_data = pivoted_data[["EPS","ROE","SAL","NET","EBI","EBT","GPS","DPS","BPS","NAV","OFTIC","PENDS","MEAN","STDEV", "BPS", "CPS", "PRE", "NDT"]]

In [15]:
len(company_list)

276

In [16]:
divisors = [i for i in range(1, len(company_list) + 1) if len(company_list) % i == 0]
divisors

[1, 2, 3, 4, 6, 12, 23, 46, 69, 92, 138, 276]

In [17]:
pivoted_data[pivoted_data["OFTIC"] == "ZTS"].sort_values(by="PENDS")

,Date,OFTIC,PENDS,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,...,commodity_trade_Close_quarterly_return,C_Discretionary_Close_quarterly_return,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return


In [18]:
# pivoted_data.reset_index(inplace=True, drop=True)
# pivoted_data

In [19]:
stock_symbol = "PGR"
stock = pivoted_data[pivoted_data["OFTIC"] == stock_symbol].sort_values(by="PENDS")

stock_features_dict = {}
for column in stock.columns:
    stock_features_dict[column] = stock[column]

trace = go.Scatter(x=stock_features_dict["PENDS"], y=stock_features_dict["EPS"], mode="lines+markers", name=f"{stock_symbol}: EPS - PENDS")

layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=trace, layout=layout)
pyo.iplot(fig)

In [20]:
pivoted_data[pivoted_data["OFTIC"]=="TROW"][["OFTIC","EPS","PENDS"]].sort_values(by=["OFTIC", "PENDS"])

,OFTIC,EPS,PENDS
12444,TROW,0.76,2011-06-30
12445,TROW,0.71,2011-09-30
12446,TROW,0.73,2011-12-31
12447,TROW,0.75,2012-03-31
12448,TROW,0.79,2012-06-30
12449,TROW,0.87,2012-09-30
12450,TROW,0.88,2012-12-31
12451,TROW,0.91,2013-03-31
12452,TROW,0.92,2013-06-30
12453,TROW,1.00,2013-09-30


In [21]:
pivoted_data[["EPS","OFTIC","PENDS"]].sort_values(by=["OFTIC", "PENDS"])

,EPS,OFTIC,PENDS
0,0.2782,AAPL,2011-06-30
1,0.2518,AAPL,2011-09-30
2,0.4954,AAPL,2011-12-31
3,0.4393,AAPL,2012-03-31
4,0.3329,AAPL,2012-06-30
...,...,...,...
14055,1.0800,ZION,2018-12-31
14056,1.0400,ZION,2019-03-31
14057,0.9900,ZION,2019-06-30
14058,1.1700,ZION,2019-09-30


In [22]:
x_values = pivoted_data.drop(columns=["OFTIC","PENDS","Date","Sector","numeric_sector"])
x_values
cols_to_move = ['OFTIC', 'PENDS', 'Date', 'Sector', 'numeric_sector']
pivoted_data = pivoted_data[[col for col in pivoted_data if col not in cols_to_move] + cols_to_move]
pivoted_data

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return,OFTIC,PENDS,Date,Sector,numeric_sector
0,17.869831,1.698451,2.6718,0.422900,777.000000,0.209015,0.00,0.230530,9379.0,0.505000,...,7.308964,-1.141483,-1.381423,-1.624192,5.051769,AAPL,2011-06-30,2011-06-30,Information Technology,3
1,17.834875,3.131120,2.9872,0.397600,1645.000000,0.258659,0.00,0.303239,8710.0,0.551793,...,-10.695185,-21.535988,-8.171207,-25.425447,0.418158,AAPL,2011-09-30,2011-09-30,Information Technology,3
2,21.771714,3.087308,3.4500,0.665700,1321.000000,0.249095,0.00,0.274352,17340.0,0.885858,...,9.328708,15.503083,7.838985,14.100815,7.019633,AAPL,2011-12-31,2011-12-31,Information Technology,3
3,26.426070,2.380069,3.9154,0.528200,1457.000000,0.153558,0.00,0.192177,15384.0,0.626165,...,8.417417,10.874069,18.506872,10.358213,-2.612559,AAPL,2012-03-31,2012-03-31,Information Technology,3
4,27.652509,3.109565,4.2582,0.384300,2056.000000,0.204900,0.00,0.246704,11573.0,0.666070,...,1.063541,-4.676644,-4.741380,-4.544226,5.565071,AAPL,2012-06-30,2012-06-30,Information Technology,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,53.000000,4.600140,37.3900,1.760000,45.779944,0.504142,0.30,0.519314,297.0,0.587800,...,-9.080399,-17.844385,-17.722026,-12.791300,0.512814,ZION,2018-12-31,2018-12-31,Financials,5
14056,52.800000,3.001754,41.5751,-0.210000,41.655270,0.300673,0.30,0.245076,278.0,0.327376,...,6.057101,16.488114,19.393354,9.857481,9.920635,ZION,2019-03-31,2019-03-31,Financials,5
14057,49.574468,4.422004,42.9480,1.380000,44.643278,0.516266,0.30,0.489922,277.0,0.540476,...,0.970027,3.185392,5.459461,5.405405,2.509890,ZION,2019-06-30,2019-06-30,Financials,5
14058,51.193548,3.780823,40.7500,1.583422,44.768947,0.615809,0.34,0.638738,298.0,0.654236,...,-2.709415,0.271247,3.190669,-0.512819,8.569506,ZION,2019-09-30,2019-09-30,Financials,5


In [23]:
corr = x_values.corr()

fig = go.Figure(data=go.Heatmap(
                   z=corr.values,
                   x=corr.columns,
                   y=corr.columns,
                   text=corr.round(2).values,
                   texttemplate="%{text}",
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=900,  # or any width you desire
    height=800,  # or any height you desire
)

In [24]:
linked = sch.linkage(sch.distance.pdist(corr), method='ward')
cluster_order = sch.dendrogram(linked, no_plot=True)['leaves']

# Reorder the correlation matrix
correlation_matrix_ordered = corr.iloc[cluster_order, cluster_order]

fig = go.Figure(data=go.Heatmap(
                   z=correlation_matrix_ordered.values,
                   x=correlation_matrix_ordered.columns,
                   y=correlation_matrix_ordered.columns,
                   text=correlation_matrix_ordered.round(2).values,
                   texttemplate="%{text}",
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

# Update the layout
fig.update_layout(
    title='Correlation Matrix',
    xaxis_title="Variables",
    yaxis_title="Variables",
    xaxis=dict(side='bottom'),
    yaxis=dict(autorange='reversed'),
    width=900,  # or any width you desire
    height=800,  # or any height you desire
)

fig.show()

In [25]:
X_ = pivoted_data.drop(columns=["EPS"])
y_ = pivoted_data['EPS']
y_

0        0.2782
1        0.2518
2        0.4954
3        0.4393
4        0.3329
          ...  
14055    1.0800
14056    1.0400
14057    0.9900
14058    1.1700
14059    0.9700
Name: EPS, Length: 9660, dtype: float64

In [26]:
company_dict = {}

for company in tqdm(pivoted_data["OFTIC"].unique()):
    comp_data = pivoted_data[pivoted_data["OFTIC"] == company].sort_values(by='PENDS')
    X = comp_data.drop(columns=["EPS", "Sector","PENDS","OFTIC","Date"])
    y = comp_data['EPS']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    X_train_df = X_train
    X_test_df = X_test
    y_train_df = y_train
    y_test_df = y_test

    for column in X_train.columns:
        # if column in ["PENDS","OFTIC","Date"]:
        #     continue
        if column == "numeric_sector":
            continue

        scaler = MinMaxScaler()

        X_train_scaled = scaler.fit_transform(X_train[[column]].values)
        X_train_df[column] = X_train_scaled
            
        X_test_scaled = scaler.transform(X_test[[column]].values)
        X_test_df[column] = X_test_scaled

    scaler_y = MinMaxScaler()

    y_train_scaled = pd.DataFrame(scaler_y.fit_transform(y_train.values.reshape(-1, 1)), columns=['EPS'], index=y_train.index)
    y_train_df = y_train_scaled
            
    y_test_scaled = pd.DataFrame(scaler_y.transform(y_test.values.reshape(-1, 1)), columns=['EPS'], index=y_test.index)
    y_test_df = y_test_scaled

    X_train_df["lagged_EPS"] = y_train_df
    X_test_df["lagged_EPS"] = y_test_df

    company_dict[company] = {"X_train": X_train_df, "X_test": X_test_df, "y_train": y_train_df, "y_test": y_test_df, "scaler_y": scaler_y}


  0%|          | 0/276 [00:00<?, ?it/s]

In [27]:
company_dict["PGR"]["X_test"]

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return,numeric_sector,lagged_EPS
9973,1.071070,0.302241,1.138858,0.968741,0.732330,0.913012,0.000000,0.862455,0.714346,0.804487,...,0.251749,0.864275,0.437380,0.515463,0.484770,0.538344,0.693463,0.445998,5,0.871560
9974,1.232931,0.099889,1.312640,1.136925,0.738867,1.218710,0.000000,1.139008,0.993442,1.087978,...,0.566433,0.545328,0.600558,0.953648,0.836612,0.622477,0.637155,0.375589,5,1.036697
9975,1.045968,0.365504,0.998169,0.957238,1.365191,1.079371,0.000000,1.093243,0.835560,0.985656,...,0.076821,-0.047689,0.211407,0.072120,0.099668,-0.358002,0.319639,0.337380,5,0.954128
9976,1.355125,0.274253,1.360514,1.014460,0.950904,0.416721,0.300325,0.387881,1.248482,0.482935,...,0.838735,0.929415,0.694981,0.650111,1.026594,1.033229,0.892645,0.778369,5,1.036697
9977,1.320314,0.452580,1.610092,1.077870,1.792527,0.979763,0.067114,0.962517,0.799854,0.980050,...,0.512132,0.461833,0.681903,0.455872,0.667441,0.510931,0.780009,0.430992,5,1.119266
9978,1.278996,0.207665,1.770002,1.149039,0.986787,1.276714,0.067114,1.245461,1.185465,1.283922,...,0.618266,0.377041,0.549573,0.315381,0.588763,0.425888,0.630280,0.715035,5,1.036697
9979,1.409959,0.050862,1.677857,0.968847,1.056922,0.950983,0.067114,0.984237,1.025625,0.997427,...,0.467675,0.586866,0.739691,0.915764,0.714989,0.824818,0.783228,0.304654,5,0.899083


In [28]:
company_dict["TROW"]["X_train"]

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,C_Staples_Close_quarterly_return,Energy_Close_quarterly_return,Financials_Close_quarterly_return,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return,numeric_sector,lagged_EPS
12444,0.094055,0.093334,0.032658,0.593023,0.208539,0.251689,0.000000,0.267056,0.113715,0.084615,...,0.553415,0.415602,0.374808,0.697910,0.550621,0.254508,0.602163,0.550142,5,0.047170
12445,0.000000,0.052220,0.000000,0.682171,0.198291,0.473431,0.000000,0.435979,0.005513,0.023077,...,0.114769,0.000000,0.000000,0.010463,0.000000,0.000000,0.000000,0.332943,5,0.000000
12446,0.087462,0.100018,0.049428,0.205426,0.465784,0.433615,0.000000,0.393485,0.000000,0.000000,...,0.794207,1.000000,0.743001,0.775029,1.000000,0.600125,1.000000,0.642385,5,0.018868
12447,0.153808,0.012053,0.115627,0.577519,0.000000,0.120564,0.013575,0.106131,0.077877,0.000000,...,0.577455,0.645411,1.000000,0.740234,0.875023,1.000000,0.905314,0.190879,5,0.037736
12448,0.147095,0.054947,0.051194,0.534884,0.139073,0.358470,0.013575,0.285772,0.118539,0.000000,...,0.443646,0.366688,0.352466,0.459443,0.455177,0.128563,0.528287,0.574203,5,0.075472
12449,0.197957,0.162969,0.195066,0.740310,0.163355,0.555681,0.013575,0.465401,0.225017,0.014902,...,0.491400,0.814483,0.662573,0.630793,0.646533,0.580276,0.751507,0.237308,5,0.150943
12450,0.237328,0.127765,0.179178,0.255814,0.209713,0.477393,0.466063,0.405140,0.232943,0.102236,...,0.228173,0.484243,0.632134,0.395993,0.682694,0.065555,0.694129,0.123988,5,0.160377
12451,0.310290,0.276269,0.256852,0.864341,0.245033,0.164767,0.031674,0.131730,0.272226,0.096496,...,1.000000,0.824606,0.766053,1.000000,0.856411,0.490786,0.753780,0.874442,5,0.188679
12452,0.312487,0.412637,0.322168,0.585271,0.154525,0.366330,0.031674,0.300360,0.362509,0.171945,...,0.337496,0.520387,0.669756,0.556774,0.638333,0.345915,0.589659,0.137113,5,0.198113
12453,0.346015,0.078710,0.395428,0.728682,0.163355,0.699661,0.031674,0.644527,0.455892,0.425983,...,0.364506,0.697205,0.570106,0.657827,0.820146,0.483967,0.884706,0.280954,5,0.273585


In [29]:
company_dict["PGR"]["y_test"].iloc[1]["EPS"]

1.036697247706422

In [30]:
X.columns

Index(['MEAN', 'STDEV', 'BPS', 'CPS', 'CPX', 'CSH', 'DPS', 'EBG', 'EBI', 'EBS',
       'EBT', 'ENT', 'FFO', 'GPS', 'GRM', 'NAV', 'NDT', 'NET', 'OPR', 'PRE',
       'ROA', 'ROE', 'SAL', 'Real_Estate_Index_Price', 'VIX_Close',
       'Gold_Close', 'Three_Month_Yield', 'Brent_Close', 'Hrc_close',
       'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close',
       'Energy_Close', 'Financials_Close', 'Health_care_Close',
       'industrials_Close', 'information_Close', 'materials_Close',
       'utilities_Close', 'Real_Estate_Index_Price_quarterly_return',
       'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return',
       'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return',
       'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return',
       'C_Discretionary_Close_quarterly_return',
       'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return',
       'Financials_Close_quarterly_return',
       'Health_care_Close_quarterl

In [31]:
# Custom Dataset
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, device=device):
        self.X = X.clone().detach().to(torch.float)
        self.y = y.clone().detach().to(torch.float)

    def __len__(self):
        return len(self.X) 

    def __getitem__(self, idx):
        # Ensure idx is within the valid range
        if idx > len(self.X):
            raise IndexError("Index out of bounds")

        X_window = self.X[idx]
        y_target = self.y[idx]  

        return X_window.clone().detach().to(torch.float).to(device), y_target.clone().detach().to(torch.float).to(device)

In [32]:
company_dict["TROW"]["X_train"]["lagged_EPS"]


12444    0.047170
12445    0.000000
12446    0.018868
12447    0.037736
12448    0.075472
12449    0.150943
12450    0.160377
12451    0.188679
12452    0.198113
12453    0.273585
12454    0.330189
12455    0.320755
12456    0.396226
12457    0.386792
12458    0.443396
12459    0.396226
12460    0.500000
12461    0.330189
12462    0.433962
12463    0.415094
12464    0.669811
12465    0.537736
12466    0.745283
12467    0.669811
12468    0.745283
12469    0.801887
12470    0.886792
12471    1.000000
Name: lagged_EPS, dtype: float64

In [33]:
X_train = []
y_train = []

X_test = []
y_test = []

time_steps = 4
print(len(company_dict[company]["X_train"]))
print((len(company_dict[company]["X_test"])))

for company in company_list:
    company_dict[company]["y_train"]["Type"] = "Train"
    company_dict[company]["y_test"]["Type"] = "Test"

    comp_df_X = pd.concat([company_dict[company]['X_train'], company_dict[company]['X_test']], axis=0)
    comp_df_y = pd.concat([company_dict[company]['y_train'], company_dict[company]['y_test']], axis=0)

    for i in range((len(comp_df_X)) - time_steps):

        if comp_df_y.iloc[i + time_steps]["Type"] == "Train":
            X_train.append(comp_df_X.iloc[i : (i + time_steps)])
            y_train.append(comp_df_y.iloc[i + time_steps]["EPS"])

        elif comp_df_y.iloc[i + time_steps]["Type"] == "Test":
            X_test.append(comp_df_X.iloc[i : (i + time_steps)])
            y_test.append(comp_df_y.iloc[i + time_steps]["EPS"])

    # for i in range((len(company_dict[company]["X_train"])) - time_steps):

    #     X_train.append(company_dict[company]["X_train"].iloc[i : (i + time_steps)])
    #     y_train.append(company_dict[company]["y_train"][i+time_steps])

    # for i in range((len(company_dict[company]["X_test"])) - time_steps):

    #     X_test.append(company_dict[company]["X_test"].iloc[i : (i+ time_steps)])
    #     y_test.append(company_dict[company]["y_test"][i+time_steps])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

28
7


In [34]:
y_train

array([0.11252949, 0.08019981, 0.33495213, ..., 0.60215054, 0.68817204,
       1.        ])

In [35]:
X_test[6][3]

array([1.52052514, 2.0932573 , 0.57259496, 0.70575221, 0.58513751,
       1.35513259, 1.22222222, 1.29870939, 0.39370303, 0.59279778,
       0.48320348, 1.64843955, 1.26838408, 0.70167892, 0.10219675,
       0.29842215, 0.90724788, 0.52544264, 0.53023757, 0.40221503,
       0.20669483, 0.90618627, 0.59593822, 1.11973536, 0.20119581,
       0.57886804, 1.0867052 , 0.27453271, 0.28205128, 0.69148982,
       1.2921872 , 1.16636059, 0.01659059, 1.0049137 , 1.14622172,
       1.04219589, 1.36131041, 0.9252487 , 1.59734281, 0.97190903,
       0.25889315, 0.69890595, 0.25336927, 0.47416963, 0.37197563,
       0.20511715, 0.50460477, 0.61826574, 0.37704129, 0.54957345,
       0.3153813 , 0.58876301, 0.42588808, 0.63028039, 0.71503503,
       3.        , 0.70167892])

In [36]:
y_test[5]

0.7016789232690439

In [37]:
X_train.shape

(6624, 4, 57)

In [38]:
X_train.shape

(6624, 4, 57)

In [39]:
y_test.shape

(1932,)

In [40]:
y_test

array([0.46232829, 0.66005273, 1.10059664, ..., 0.89247312, 1.08602151,
       0.87096774])

In [41]:
X_test.shape

(1932, 4, 57)

In [42]:
X_train.shape

(6624, 4, 57)

In [43]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float, device=device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float, device=device)

train_data = RollingWindowDataset(X_train_tensor, y_train_tensor, device=device)
test_data = RollingWindowDataset(X_test_tensor, y_test_tensor, device=device)

print(test_data.__getitem__(0)[1])
print(test_data.__getitem__(1)[0])


tensor(0.4623, device='cuda:0')
tensor([[0.9412, 0.6188, 0.9148, 0.4004, 0.9035, 1.0000, 1.0000, 1.0000, 0.2511,
         0.2742, 0.3213, 0.9125, 1.0000, 0.3687, 0.0962, 0.9132, 0.1532, 0.3043,
         0.4217, 0.2810, 0.0549, 0.3080, 0.4050, 0.4033, 0.0000, 0.3073, 0.6127,
         0.2367, 0.4444, 0.7656, 0.8313, 0.8931, 0.2397, 0.8881, 0.9814, 0.8995,
         0.8489, 0.8803, 1.0000, 0.7087, 0.1488, 0.6628, 0.2940, 0.9171, 0.2943,
         0.1143, 0.4784, 0.2677, 0.6872, 0.6252, 0.5388, 0.6960, 0.6063, 0.7838,
         0.4117, 3.0000, 0.3687],
        [0.9386, 0.7979, 1.0000, 0.9469, 0.5948, 0.5943, 1.0000, 0.5320, 1.0000,
         1.0000, 1.0000, 1.0000, 0.5742, 1.0000, 0.1442, 1.0000, 0.0000, 1.0000,
         0.8406, 1.0000, 0.3351, 0.9053, 1.0000, 0.4785, 0.0457, 0.3398, 0.8035,
         0.3457, 0.5791, 0.6857, 0.9609, 1.0000, 0.3306, 1.0000, 1.0000, 1.0000,
         0.9648, 1.0000, 0.9811, 0.7957, 0.2997, 0.6308, 0.3747, 0.8576, 0.6245,
         0.1207, 0.7922, 0.6006, 0.6881, 0.

In [44]:
# class VanillaLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
#         super(VanillaLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn, self.cn = None, None

#         self.lstm = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
#                             dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn = None
#         self.cn = None

#     def forward(self, x):
#         if self.hn == None or self.cn == None:
#             self.hn, self.cn = self.init_hidden(x.size(0))
        
#         else:
#             self.hn, self.cn = self.hn.detach(), self.cn.detach()
            
#         if self.hn.size(1) > x.size(0):
#             self.hn = self.hn[:, -x.size(0):, :].contiguous()
#             self.cn = self.cn[:, -x.size(0):, :].contiguous()

#         # Forward propagate LSTM
#         out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))

#         out = self.dropout(out[:, -1, :])  # Add dropout

#         out = self.fc(out)

#         return out

In [45]:
class VanillaLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
        super(VanillaLSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.layer_size = layer_size
        self.hn = None
        self.cn = None

        self.lstm = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
                            dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
        self.dropout = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(self.hidden_size, output_dim)

    def init_hidden(self, batch_size):
        # Initialize hidden and cell states with zeros
        h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
        return (h0, c0)
    
    def reset_hidden(self):
        self.hn = None
        self.cn = None

    def forward(self, x):

        if self.hn == None or self.cn == None:
            self.hn, self.cn = self.init_hidden(x.size(0))

        else:
            self.hn, self.cn = self.hn.detach(), self.cn.detach()

            last_hn, last_cn = self.hn[:,-1:,:], self.cn[:, -1:, :]

            self.hn, self.cn = last_hn.repeat(1, x.size(0), 1), last_cn.repeat(1, x.size(0), 1)
        
    
        # Forward propagate LSTM
        out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))

        out = self.dropout(out[:, -1, :])  # Add dropout

        out = self.fc(out)

        return out

In [46]:
# class VanillaLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
#         super(VanillaLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn, self.cn = None, None

#         self.lstm = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
#                             dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn = None
#         self.cn = None

#     def forward(self, x):
#         # self.hn, self.cn = self.init_hidden(x.size(0))
#         if self.hn == None or self.cn == None:
#             self.hn, self.cn = self.init_hidden(1)
#         else:
#             self.hn, self.cn = self.hn.detach(), self.cn.detach()

#         x_longer = x.view(1,x.shape[0]*x.shape[1],x.shape[2])

#         out_longer, (self.hn, self.cn) = self.lstm(x_longer, (self.hn, self.cn))

#         out = out_longer.view(x.shape[0],x.shape[1],out_longer.shape[2])

#         out = self.dropout(out[:,-1,:])
#         out = self.fc(out)

#         return out


In [47]:
def find_supported_splits(comp_size):
    supported_splits = []
    for n_splits in range(1, comp_size + 1):
        if comp_size % n_splits != 0:
            continue
        
        supported_splits.append(n_splits)
    return supported_splits


def custom_time_series_folds(data, n_splits):

    total_size = len(data)
    comp_size = total_size // len(company_list)
    comp_fold_size = comp_size//n_splits

    if comp_size % n_splits != 0:
        supported_splits = find_supported_splits(comp_size)
        print(supported_splits)
        print(f"fold_size: {comp_fold_size} comp_size: {comp_size}")
        raise ValueError("Fold size must be divisible by the number of companies.")

    accumulated_train_idx = []     

    for i in range(n_splits-1):
        current_fold_val_idx = []
        current_fold_train_idx = []

        for j in range(len(company_list)):

            start_idx = j * comp_size
            val_start_idx = start_idx + (i+1) * comp_fold_size 
        
            end_idx = val_start_idx + comp_fold_size
        
            current_comp_train_idx = list(range(start_idx, val_start_idx))
            current_fold_train_idx.extend(current_comp_train_idx)  
        
            val_idx = list(range(val_start_idx, end_idx))
            current_fold_val_idx.extend(val_idx)  

        print(f"train: {current_fold_train_idx}")
        print(f"test: {current_fold_val_idx}")
        yield current_fold_train_idx, current_fold_val_idx

In [48]:
class ModelActioner:

    def __init__(self, train_data, test_data, device):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.model = None
        self.optimizer = None
        self.criterion = nn.MSELoss()

    def custom_tscv(self, config, trial):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["layer_size"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//5
        gamma = config["gamma"]

        suffle = False

        num_of_fold = 4

        fold_results = []

        for fold, (train_idx, val_idx) in enumerate(custom_time_series_folds(self.train_data, num_of_fold)):
            print(f"Fold: {fold+1}/{num_of_fold}")

            train_subset = Subset(self.train_data, train_idx)
            val_subset = Subset(self.train_data, val_idx)

            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=suffle)
            val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=suffle)

            self.model = VanillaLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

            self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
            scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True) 

            for epoch in range(epochs):
                print('epochs {}/{}'.format(epoch+1,epochs))

                self.model.reset_hidden()

                running_loss = 0.0
                total_sample_train = 0

                self.model.train()

                for batch_idx, (data, target) in enumerate(train_loader):
                    data, target = data.to(self.device), target.to(self.device)
                    target = target.view(-1,1) 

                    self.optimizer.zero_grad()
                    preds = self.model(data)

                    loss = self.criterion(preds, target)
                    loss.backward()
                    self.optimizer.step() # Update model params

                    running_loss += loss.item() * data.size(0)
                    total_sample_train += data.size(0)

                train_loss = running_loss/total_sample_train

                self.model.eval()
                val_running_loss = 0.0
                total_sample_val = 0

                with torch.no_grad():

                    for batch_idx, (data, target) in enumerate(val_loader):
                        data, target = data.to(self.device), target.to(self.device)
                        target = target.view(-1,1)

                        preds = self.model(data)
                        loss = self.criterion(preds, target)

                        val_running_loss += loss.item() * data.size(0)
                        total_sample_val += data.size(0)
                
                val_loss = val_running_loss/total_sample_val
                fold_results.append(val_loss)
                scheduler.step(val_loss)
                
                unique_step = fold * epochs + epoch
                trial.report(val_loss, unique_step)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                current_lr = self.optimizer.param_groups[0]['lr']

                print(f'Current Learning Rate: {current_lr}')
                print(f"train_loss: {train_loss}, val_loss: {val_loss}")
                
        mean_val_loss = np.mean(fold_results)
        print(f"Mean validation loss: {mean_val_loss}")
        return mean_val_loss


                    
    def train(self, config):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["layer_size"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//5
        gamma = config["gamma"]

        self.model = VanillaLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

        # Update optimizer with updated lr
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate, weight_decay=weight_decay)

        # Creating data loader
        train_loader = DataLoader(dataset=self.train_data, batch_size=batch_size, shuffle=False)

        scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True)  

        # Training Loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))

            self.model.reset_hidden()
            
            running_loss = 0.0
            total_sample_train = 0

            self.model.train()

            for batch_idx, (data, target) in enumerate(train_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)  
                # print(data)

                self.optimizer.zero_grad()
                preds = self.model(data)

                loss = self.criterion(preds, target)
                loss.backward()
                self.optimizer.step() # Update model params

                running_loss += loss.item() * data.size(0)
                total_sample_train += data.size(0)

            train_loss = running_loss/total_sample_train
            scheduler.step(train_loss)
            current_lr = self.optimizer.param_groups[0]['lr']

            print(f'Current Learning Rate: {current_lr}')
            print(f"train_loss: {train_loss}")
        
        return self.model
            
    
    def test(self, config):
        batch_size = config["batch_size"]
        all_preds = []

        test_loader = DataLoader(dataset=self.test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0
        total_sample = 0

        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(test_loader):

                data, target = data.to(self.device), target.to(self.device)
                target = target.view(-1,1)
                
                preds = self.model(data)
                loss = self.criterion(preds, target)

                running_loss += loss.item() * data.size(0)
                total_sample += data.size(0)

                all_preds.extend(preds.cpu().numpy())

            test_loss = running_loss/total_sample
            print(f"test_loss: {test_loss}")

        return all_preds
    


In [49]:
def objective(trial):
    config = {
        "batch_size": trial.suggest_int("batch_size", 22, 100),
        "epochs": trial.suggest_int("epochs", 50, 400),
        "hidden_size": trial.suggest_int("hidden_size", 50, 400),
        "layer_size": trial.suggest_int("layer_size", 1, 6),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1),
        "dropout_prob": trial.suggest_float("dropout_prob", 0.1, 0.30),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True),
        # "lr_step_size": trial.suggest_int("lr_step_size", 10, 100), 
        "gamma": trial.suggest_float("gamma", 1e-4, 1e-1)
    }

    trainer = ModelActioner(train_data, test_data, device)

    val_loss = trainer.custom_tscv(config, trial)

    return val_loss

In [50]:
study_name = "Vanilla-LSTM-Tunner"
storage_url = "sqlite:///db.sqlite3"

storage = optuna.storages.RDBStorage(url=storage_url)

# Check if the study exists
study_names = [study.study_name for study in optuna.study.get_all_study_summaries(storage=storage)]
if study_name in study_names:
    # Delete the study if it exists
    print(f"Deleting study '{study_name}'")
    optuna.delete_study(study_name=study_name, storage=storage_url)
else:
    print(f"Study '{study_name}' does not exist in the storage.")
    
study = optuna.create_study(direction='minimize', 
                            storage=storage_url, 
                            sampler=TPESampler(),
                            pruner=optuna.pruners.SuccessiveHalvingPruner(
                            min_resource=3,  # Minimum amount of resource allocated to a trial
                            reduction_factor=2,  # Reduction factor for pruning
                            min_early_stopping_rate=3 # Minimum early-stopping rate
                            ),
                            study_name=study_name,
                            load_if_exists=False)

pbar = tqdm(total=20, desc='Optimizing', unit='trial')

def callback(study, trial):
    # Update the progress bar
    pbar.update(1)
    pbar.set_postfix_str(f"Best Value: {study.best_value:.4f}")

study.optimize(objective, n_trials=20, callbacks=[callback])
pbar.close()

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value:', trial.value)
print('Params:')
for key, value in trial.params.items():
    print(f'{key}: {value}')

In [51]:
params = {
    "batch_size": 98,
    "epochs": 261,
    "hidden_size": 384,
    "layer_size": 1,
    "learning_rate":  0.020047744221561352,
    "dropout_prob": 0.10660045126151484,
    "weight_decay": 6.48234322524607e-05,
    "gamma": 0.02600814963458074
}

model = ModelActioner(train_data=train_data,test_data=test_data,device=device)
model.train(params)

epochs 1/261
Current Learning Rate: 0.020047744221561352
train_loss: 1.1976210705014756
epochs 2/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.05408896523829244
epochs 3/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.05445314457264373
epochs 4/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.04966333437911212
epochs 5/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.04790715824200767
epochs 6/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.04699258504048494
epochs 7/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.04665714228055597
epochs 8/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.046440903506250734
epochs 9/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.046230418800493334
epochs 10/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.044169486848443996
epochs 11/261
Current Learning Rate: 0.020047744221561352
train_loss: 0.043425124509210565
epochs 12/261
Cu

KeyboardInterrupt: 

------DENEME--------

In [ ]:

# params = {
#     "batch_size": 98,
#     "epochs": 261,
#     "hidden_size": 384,
#     "layer_size": 1,
#     "learning_rate":  0.020047744221561352,
#     "dropout_prob": 0.10660045126151484,
#     "weight_decay": 6.48234322524607e-05,
#     "gamma": 0.02600814963458074
# }
# # Assuming `train_data` and `test_data` are already defined and loaded
# model_actioner = ModelActioner(train_data=train_data, test_data=test_data, device=device)

# # Train the model using the specified parameters
# trained_model = model_actioner.train(params)


In [ ]:
# # Test the model and get predictions
# predictions = model_actioner.test(params)
# predictions = np.array(predictions)
# # Optionally print or analyze the predictions
# print("Predictions:", predictions)
# preds = predictions

----------DENEME-------------

In [ ]:
y_test = y_test.reshape(-1,1)
print(len(y_test))
y_test[:4]

1932


array([[0.46232829],
       [0.66005273],
       [1.10059664],
       [0.50395449]])

In [ ]:
preds = model.test(params)

test_loss: 0.1620770150768584


In [ ]:
preds = np.array(preds)
preds

array([[0.5213355 ],
       [0.5841743 ],
       [1.0392433 ],
       ...,
       [0.7061113 ],
       [0.76242524],
       [0.72938406]], dtype=float32)

In [ ]:
preds_inverse = []
y_true_inverse = []

idx = 0
data_size = len(y_test)//len(company_list)
print(data_size)

preds_comp_data = []
y_true_comp_data = pd.DataFrame()

for company in company_list:
    scaler = company_dict[company]["scaler_y"]
    preds_inverse.append(scaler.inverse_transform(preds[idx:idx+data_size]).flatten())
    y_true_inverse.append(scaler.inverse_transform(y_test[idx:idx+data_size]).flatten())

    for pred in preds_inverse:
        preds_comp_data.append({"Prediction": pred, "Company": company})

    idx += data_size



# for company in company_list:
#     preds_inverse.append(np.array(preds).flatten())
#     y_true_inverse.append(np.array(y_test).flatten())

comp_preds_inverse = np.array(preds_inverse)
comp_true_inverse = np.array(y_true_inverse)

preds_inverse = np.array(preds_inverse).flatten()
y_true_inverse = np.array(y_true_inverse).flatten()
print(y_true_inverse.shape)
print(preds_inverse.shape)



mse = mean_squared_error(y_true_inverse, preds_inverse)
mape = mean_absolute_percentage_error(y_true_inverse, preds_inverse)*100
mae = mean_absolute_error(y_true_inverse, preds_inverse)
r2 = r2_score(y_true_inverse,preds_inverse)

print(f"MAPE Score: %{mape:.2f}")
print(f"MSE Score: {mse:.2f}")
print(f"MAE Score: {mae:.2f}")
print(f"R_2 Score: {r2:.2f}")

7
(1932,)
(1932,)
MAPE Score: %79849460989192.81
MSE Score: 1.42
MAE Score: 0.43
R_2 Score: 0.88


In [ ]:
y_true_inverse

array([0.585 , 0.7275, 1.045 , ..., 0.99  , 1.17  , 0.97  ])

In [ ]:
preds_inverse = np.array(preds_inverse).flatten()
preds_inverse

array([0.62752646, 0.67281437, 1.0007826 , ..., 0.81668353, 0.8690555 ,
       0.83832717], dtype=float32)

In [ ]:
len(y_test)

1932

In [ ]:
y_test.flatten()
x_indices = np.arange(len(y_true_inverse))
preds = np.array(preds)
preds

array([[0.5213355 ],
       [0.5841743 ],
       [1.0392433 ],
       ...,
       [0.7061113 ],
       [0.76242524],
       [0.72938406]], dtype=float32)

In [ ]:
len(y_true_inverse)

1932

In [ ]:
trace_true = go.Scatter(x=x_indices, y=y_true_inverse, mode="lines", name=f"All Stocks True: EPS - PENDS")
trace_preds = go.Scatter(x=x_indices, y=preds_inverse, mode="lines", name=f"All Stocks Preds: EPS - PENDS")


layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=[trace_true, trace_preds], layout=layout)
fig.show()

In [ ]:
x_indices = [["2018-Q2","2018-Q3", "2018-Q4" ,"2019-Q1", '2019-Q2', '2019-Q3', '2019-Q4']] * len(company_list)

In [ ]:
stock_symbol = "Multiple Companies"

# Initialize the figure with layout
layout = go.Layout(
    title=f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=700,
    width=1400
)
fig = go.Figure(layout=layout)

# Variables to keep track of the extended x-axis and y-axis data
extended_x_indices = []
all_true_y = []
all_preds_y = []

# Adjust x_indices and concatenate y data for each company
for i, company in enumerate(company_list):
    # Extend x_indices with company identifier to differentiate time points between companies
    extended_x_indices += [f"{x} {company}" for x in x_indices[i]]
    all_true_y += list(comp_true_inverse[i].flatten())
    all_preds_y += list(comp_preds_inverse[i].flatten())

# Add traces for the concatenated true and predicted EPS values
trace_true = go.Scatter(x=extended_x_indices, y=all_true_y, mode="lines+markers", name="All Companies True: EPS - PENDS")
trace_preds = go.Scatter(x=extended_x_indices, y=all_preds_y, mode="lines+markers", name="All Companies Preds: EPS - PENDS")

# Add traces to the figure
fig.add_trace(trace_true)
fig.add_trace(trace_preds)

# Show the figure
fig.show()

In [ ]:
pivoted_data[pivoted_data["OFTIC"]=="AAP"]

,MEAN,STDEV,BPS,CPS,CPX,CSH,DPS,EBG,EBI,EBS,...,Health_care_Close_quarterly_return,industrials_Close_quarterly_return,information_Close_quarterly_return,materials_Close_quarterly_return,utilities_Close_quarterly_return,OFTIC,PENDS,Date,Sector,numeric_sector
